In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split

from keras.models import Model,Sequential
from keras.layers import Input,Dense,LSTM,Bidirectional
from keras import backend as K
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM 
from sklearn.preprocessing import MinMaxScaler

In [2]:
ssss= pd.read_csv('./samsung_data.csv',index_col=0)
ssss

,open,high,low,close,volume,prev,top_1,top_2,top_3,top_4,...,top_11,top_12,top_13,top_14,top_15,top_16,top_17,top_18,top_19,top_20
datetime,,,,,,,,,,,,,,,,,,,,,
2018-10-29 09:20,40800,40900,40750,40850,90497,50,-0.002968,-0.001743,-0.004199,-0.001510,...,0.004022,-0.003106,0.007953,-0.005732,0.000161,-0.000106,0.002548,0.002549,-0.007805,0.000992
2018-10-29 09:56,41050,41100,41050,41100,6777,50,-0.002120,-0.002212,0.004220,-0.006379,...,0.005555,0.002577,-0.000135,0.005530,-0.003320,0.001608,0.002994,-0.003264,0.000568,-0.003713
2018-10-29 15:30,41400,41400,41400,41400,1084329,0,-0.001114,0.006571,-0.003546,0.006705,...,-0.002814,0.006001,0.002459,-0.002532,0.004581,-0.004570,-0.004465,-0.002522,0.004941,-0.003640
2018-10-30 09:30,42450,42450,42400,42400,29979,-50,-0.001114,0.006571,-0.003546,0.006705,...,0.003558,-0.003640,-0.002814,0.002459,-0.002532,0.004581,-0.004570,-0.004465,-0.002522,0.004941
2018-10-30 11:00,42600,42650,42550,42600,9637,0,0.002620,0.007649,-0.000602,-0.002864,...,0.002968,-0.004638,0.004102,-0.001880,-0.003441,-0.004402,0.004926,-0.001276,0.004873,-0.007528
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-10-26 12:03,60700,60700,60600,60600,4623,-100,0.000620,-0.003379,-0.004112,0.008734,...,0.003576,0.003997,0.002494,-0.003869,-0.006298,-0.006515,-0.002687,0.000777,-0.004781,0.007633
2020-10-26 13:28,60400,60500,60400,60400,6501,0,0.000620,-0.003379,-0.004112,0.008734,...,0.003576,0.003997,0.002494,-0.003869,-0.006298,-0.006515,-0.002687,0.000777,0.007633,-0.004781
2020-10-30 09:59,57600,57700,57500,57600,167163,0,0.000620,-0.003379,-0.004112,0.008734,...,0.003576,0.003997,0.002494,-0.003869,-0.006298,-0.006515,-0.002687,0.000777,0.007633,-0.004781


In [12]:
ssss.columns
feature_cols = ['top_1', 'top_2',
       'top_3', 'top_4', 'top_5', 'top_6', 'top_7', 'top_8', 'top_9', 'top_10',
       'top_11', 'top_12', 'top_13', 'top_14', 'top_15', 'top_16', 'top_17',
       'top_18', 'top_19', 'top_20']

train = ssss[feature_cols] 
target =ssss['close']

In [26]:
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(ssss)

df_scaled = pd.DataFrame(data=df_scaled,columns=ssss.columns)

In [27]:
df_scaled

,open,high,low,close,volume,prev,top_1,top_2,top_3,top_4,...,top_11,top_12,top_13,top_14,top_15,top_16,top_17,top_18,top_19,top_20
0,0.145194,0.149284,0.145492,0.148980,0.082598,0.625,0.000000,0.401692,0.200927,0.358774,...,0.718593,0.292183,0.916759,0.142937,0.523117,0.490015,0.608138,0.612288,0.000000,0.577298
1,0.155419,0.157464,0.157787,0.159184,0.005316,0.625,0.119776,0.371820,0.668273,0.123935,...,0.801583,0.562439,0.476011,0.777414,0.313173,0.592433,0.634839,0.277314,0.454039,0.277843
2,0.169734,0.169734,0.172131,0.171429,1.000000,0.500,0.261846,0.931310,0.237182,0.754950,...,0.348461,0.725298,0.617364,0.323230,0.789648,0.223393,0.188181,0.320063,0.691142,0.282472
3,0.212679,0.212679,0.213115,0.212245,0.026734,0.375,0.261846,0.931310,0.237182,0.754950,...,0.693479,0.266802,0.329983,0.604412,0.360705,0.769969,0.181886,0.208143,0.286455,0.828659
4,0.218814,0.220859,0.219262,0.220408,0.007956,0.500,0.789188,1.000000,0.400598,0.293448,...,0.661520,0.219353,0.706897,0.359949,0.305836,0.233432,0.750542,0.391878,0.687439,0.034967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,0.959100,0.959100,0.959016,0.955102,0.003328,0.250,0.506693,0.297439,0.205766,0.852842,...,0.694460,0.629958,0.619279,0.247878,0.133588,0.107212,0.294664,0.510171,0.163963,1.000000
676,0.946830,0.950920,0.950820,0.946939,0.005061,0.500,0.506693,0.297439,0.205766,0.852842,...,0.694460,0.629958,0.619279,0.247878,0.133588,0.107212,0.294664,0.510171,0.837100,0.209801
677,0.832311,0.836401,0.831967,0.832653,0.153368,0.500,0.506693,0.297439,0.205766,0.852842,...,0.694460,0.629958,0.619279,0.247878,0.133588,0.107212,0.294664,0.510171,0.837100,0.209801
678,0.852761,0.852761,0.852459,0.853061,0.003620,0.500,0.506693,0.297439,0.205766,0.852842,...,0.694460,0.629958,0.619279,0.247878,0.133588,0.107212,0.294664,0.510171,0.837100,0.209801


In [28]:
target = df_scaled['close']

In [33]:
train = df_scaled.drop(['close'],axis=1)

In [3]:
from sklearn.preprocessing import StandardScaler

In [4]:
scaler = StandardScaler()
scaler.fit(ssss)
ssss_scaled = scaler.transform(ssss)

In [5]:
ssss_scaled

array([[-0.92589568, -0.91537701, -0.92864069, ...,  0.5885835 ,
        -2.10546347,  0.23316766],
       [-0.88692314, -0.88424758, -0.88179068, ..., -0.87882084,
        -0.05748082, -0.96316882],
       [-0.8323616 , -0.83755343, -0.82713233, ..., -0.69155217,
         1.01199524, -0.94467307],
       ...,
       [ 1.69305852,  1.6994954 ,  1.68715154, ...,  0.14124484,
         1.67035145, -1.23499874],
       [ 1.77100358,  1.76175426,  1.76523489, ...,  0.14124484,
         1.67035145, -1.23499874],
       [ 2.05160582,  2.05748388,  2.06195162, ..., -0.14320515,
         0.2369239 ,  1.81046092]])

In [6]:
ssss_df_scaled = pd.DataFrame(data=ssss_scaled,columns=ssss.columns)

In [7]:
ssss_df_scaled

,open,high,low,close,volume,prev,top_1,top_2,top_3,top_4,...,top_11,top_12,top_13,top_14,top_15,top_16,top_17,top_18,top_19,top_20
0,-0.925896,-0.915377,-0.928641,-0.918322,0.747492,0.845552,-3.741123,0.026095,-1.065624,-0.224553,...,0.900484,-0.858504,1.698292,-1.352836,0.049482,0.239672,0.334886,0.588583,-2.105463,0.233168
1,-0.886923,-0.884248,-0.881791,-0.879351,-0.435664,0.845552,-2.823881,-0.200705,0.364995,-1.135223,...,1.290957,0.245413,0.092083,0.724836,-0.643384,0.613803,0.435856,-0.878821,-0.057481,-0.963169
2,-0.832362,-0.837553,-0.827132,-0.832587,14.792623,-0.030935,-1.735923,4.047179,-0.954641,1.311758,...,-0.841004,0.910645,0.607213,-0.762444,0.929098,-0.734295,-1.253169,-0.691552,1.011995,-0.944673
3,-0.668677,-0.674124,-0.670966,-0.676706,-0.107766,-0.907421,-1.735923,4.047179,-0.954641,1.311758,...,0.782321,-0.962177,-0.440083,0.158320,-0.486516,1.262342,-1.276974,-1.181839,-0.813380,1.237364
4,-0.645293,-0.642994,-0.647541,-0.645530,-0.395246,-0.030935,2.302441,4.568701,-0.454399,-0.477875,...,0.631952,-1.155992,0.933496,-0.642206,-0.667598,-0.697625,0.873386,-0.376956,0.995295,-1.933468
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,2.176318,2.166437,2.171268,2.160325,-0.466105,-1.783908,0.139109,-0.765438,-1.050810,1.691372,...,0.786939,0.521208,0.614191,-1.009194,-1.236058,-1.158704,-0.850508,0.141245,-1.365892,1.921878
676,2.129551,2.135307,2.140035,2.129149,-0.439565,-0.030935,0.139109,-0.765438,-1.050810,1.691372,...,0.786939,0.521208,0.614191,-1.009194,-1.236058,-1.158704,-0.850508,0.141245,1.670351,-1.234999
677,1.693059,1.699495,1.687152,1.692683,1.830959,-0.030935,0.139109,-0.765438,-1.050810,1.691372,...,0.786939,0.521208,0.614191,-1.009194,-1.236058,-1.158704,-0.850508,0.141245,1.670351,-1.234999
678,1.771004,1.761754,1.765235,1.770623,-0.461625,-0.030935,0.139109,-0.765438,-1.050810,1.691372,...,0.786939,0.521208,0.614191,-1.009194,-1.236058,-1.158704,-0.850508,0.141245,1.670351,-1.234999


In [8]:
target = ssss_df_scaled['close']

In [9]:
train = ssss_df_scaled.drop('close',axis=1)

In [10]:
X_train, X_test, y_train, y_test =\
train_test_split(
    train
    , target
    , test_size =0.2
    , random_state = 156
)


In [11]:
np.array(X_train).shape, np.array(y_train).shape

((544, 25), (544,))

In [12]:
from keras.models import Sequential
from keras.layers import Dense

layers = []
layers.append(
    Dense(
        units=128
        , input_dim=25
        , activation='relu' 
    )
)

layers.append(
    Dense(
        units= 64
        , activation = 'relu'
        , trainable=True
    )
)

layers.append(
    Dense(
        units= 32
        , activation = 'relu'
        , trainable=True
    )
)
layers.append(
    Dense(
        units= 1
        , activation = 'relu'
        , trainable=True
    )
)

model = Sequential(layers)
model.compile(loss='mse', optimizer='adam')

In [22]:
history = model.fit(X_train,y_train,epochs=100,batch_size=10)

Epoch 1/100
55/55 [==============================] - 0s 982us/step - loss: 0.3641
Epoch 2/100
55/55 [==============================] - 0s 964us/step - loss: 0.3938
Epoch 3/100
55/55 [==============================] - 0s 1ms/step - loss: 0.3640
Epoch 4/100
55/55 [==============================] - 0s 1ms/step - loss: 0.3597
Epoch 5/100
55/55 [==============================] - 0s 855us/step - loss: 0.3587
Epoch 6/100
55/55 [==============================] - 0s 855us/step - loss: 0.3585
Epoch 7/100
55/55 [==============================] - 0s 855us/step - loss: 0.3582
Epoch 8/100
55/55 [==============================] - 0s 837us/step - loss: 0.3581
Epoch 9/100
55/55 [==============================] - 0s 837us/step - loss: 0.3582
Epoch 10/100
55/55 [==============================] - 0s 818us/step - loss: 0.3581
Epoch 11/100
55/55 [==============================] - 0s 818us/step - loss: 0.3581
Epoch 12/100
55/55 [==============================] - 0s 837us/step - loss: 0.3583
Epoch 13/100
55/5

55/55 [==============================] - 0s 800us/step - loss: 0.3580
Epoch 100/100
55/55 [==============================] - 0s 800us/step - loss: 0.3580


In [18]:
y_test.shape

(136,)